<div style="background: linear-gradient(to right, #e3f2fd, #e8eaf6); /* Soft blue gradient */
             border-left: 4px solid #90caf9; /* Lighter blue border */
             padding: 18px 20px;
             border-radius: 8px;
             box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px;
             animation: slideIn 0.7s ease-out;">
 <style>
   @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-blue h1 { color: #1e88e5; margin-bottom: 12px; } /* Darker blue heading */
   .custom-div-blue p, .custom-div-blue ul { line-height: 1.6; }
   .custom-div-blue ul { padding-left: 20px; margin-top: 10px; }
   .custom-div-blue li { margin-bottom: 5px; }
   .assumption-note { background-color: #fff9c4; border-left: 3px solid #fbc02d; padding: 10px; margin: 15px 0; border-radius: 4px; font-style: italic; }
 </style>
 <div class="custom-div-blue">

  💰 Analyse de Rentabilité et Simulation (Suite DVF 2021-2025)

 **Objectif :** Ce notebook est la **quatrième étape** de notre série d'analyses (`visualisation` -> `strategique` -> `micro`). Il se focalise sur les aspects cruciaux pour la décision d'investissement d'Alexandre Dubois :
  **Estimation de la valeur** d'un bien potentiel (Besoin II.1).
  **Simulation de la rentabilité locative brute et nette** (Besoin III.2).

 **Liens vers les notebooks précédents :**
 * [Visualisation Initiale](visualisation.ipynb)
 * [Analyse Stratégique (Macro)](analyse_strategique.ipynb)
 * [Analyse Micro (Communes)](analyse_micro.ipynb)

 **Méthodologie (Simulation & Rentabilité) :**
 * **1. Configuration & Utilitaires :** Import des librairies, redéfinition des fonctions de chargement/nettoyage (`load_specific_cities_data`, `clean_dvf_data`) pour l'autonomie.
 * **2. Modélisation de la Rentabilité :** Définition de fonctions pour *simuler* le loyer (basé sur le prix/m²) et calculer les rendements brut/net en intégrant des hypothèses de charges et fiscalité. **(Hypothèse forte sur le loyer)**.
 * **3. Estimateur Interactif de Valeur et Rendement :** Création d'un outil interactif (widgets) où Alexandre peut entrer les caractéristiques d'un bien (commune, type, surface, état) et obtenir une estimation de sa valeur basée sur les transactions DVF similaires, ainsi qu'une simulation de sa rentabilité locative.
 * **4. Export des Données de Comparaison :** Possibilité d'exporter le sous-ensemble de données DVF utilisées pour une estimation spécifique.
 * **5. Conclusion & Perspectives :** Synthèse des capacités de simulation.
 </div>
 </div>


<div style="background: linear-gradient(to right, #e0f7fa, #e0f2f7); /* Soft cyan gradient */
             border-left: 4px solid #80deea; /* Lighter, thinner cyan border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-cyan h2 { color: #0097a7; margin-bottom: 12px; } .custom-div-cyan p { line-height: 1.6; } </style>
 <div class="custom-div-cyan">

 ## ⚙️ 1. Configuration et Fonctions Utilitaires

**Objectif :** Charger les bibliothèques et redéfinir les fonctions de chargement/nettoyage des notebooks précédents pour que celui-ci puisse fonctionner indépendamment si nécessaire.

 **Rappel Important :** Nous utilisons `load_specific_cities_data` (qui lit les CSV) car notre fichier Parquet `df_clean` a été optimisé pour les analyses macro et **ne contient pas** les colonnes `code_commune` et `nom_commune`, indispensables ici.

 </div>
 </div>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, FileLink
import warnings
import os
from tqdm.auto import tqdm
import gc
from datetime import datetime
import base64 # Pour l'export
import io     # Pour l'export

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
px.defaults.template = "plotly_white"

print("✅ Bibliothèques importées.")

✅ Bibliothèques importées.


<div style="background: linear-gradient(to right, #e8f5e9, #f1f8e9); /* Soft green gradient */
             border-left: 4px solid #a5d6a7; /* Lighter, thinner green border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-green h3 { color: #388e3c; margin-bottom: 12px; } .custom-div-green p { line-height: 1.6; } </style>
 <div class="custom-div-green">

 ### 🧹 1.1 Redéfinition des Fonctions de Chargement et Nettoyage

 (Identiques aux notebooks précédents)

 </div>
 </div>

In [2]:
# --- Copie de la fonction clean_dvf_data ---
def clean_dvf_data(df):
    """ Fonction de nettoyage optimisée pour les données DVF (identique aux notebooks précédents). """
    if df.empty: return pd.DataFrame()
    print("   -> Application du nettoyage standard...")
    initial_rows = len(df)
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
    cols_to_numeric = ['valeur_fonciere', 'surface_reelle_bati', 'nombre_pieces_principales']
    for col in cols_to_numeric:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
    cols_critiques = ['date_mutation', 'valeur_fonciere', 'surface_reelle_bati', 'latitude', 'longitude', 'type_local', 'code_departement', 'code_commune', 'nom_commune']
    cols_a_verifier = [col for col in cols_critiques if col in df.columns]
    df = df.dropna(subset=cols_a_verifier)
    df = df[df['valeur_fonciere'].between(1000, 20_000_000)]
    df = df[df['surface_reelle_bati'].between(10, 1000)]
    if 'nombre_pieces_principales' in df.columns: # Cette colonne peut manquer
      df = df[df['nombre_pieces_principales'].between(1, 20)]
    df = df[df['type_local'].isin(['Appartement', 'Maison'])]
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    p_01 = df['prix_m2'].quantile(0.01)
    p_99 = df['prix_m2'].quantile(0.99)
    df = df[df['prix_m2'].between(max(100, p_01), min(30000, p_99))]
    df['annee'] = df['date_mutation'].dt.year
    df['trimestre'] = df['date_mutation'].dt.to_period('Q').astype(str)
    df['mois'] = df['date_mutation'].dt.month
    final_rows = len(df)
    print(f"   -> Nettoyage terminé. {initial_rows - final_rows:,} lignes écartées. {final_rows:,} transactions valides.")
    return df

# --- Copie de la fonction load_specific_cities_data ---
BASE_CSV_PATH = './'
FILE_NAMES_CSV = [f"dvf_geolocalisees_{year}.csv" for year in range(2021, 2026)]
FILE_PATHS_CSV = [os.path.join(BASE_CSV_PATH, f) for f in FILE_NAMES_CSV if os.path.exists(os.path.join(BASE_CSV_PATH, f))]
COLS_FOR_CITY = ['date_mutation', 'valeur_fonciere', 'code_commune', 'nom_commune', 'code_departement',
                 'surface_reelle_bati', 'nombre_pieces_principales', 'type_local', 'latitude', 'longitude']
DTYPES_FOR_CITY = {'valeur_fonciere': 'float32', 'code_commune': 'str', 'nom_commune': 'str', 'code_departement': 'str',
                   'surface_reelle_bati': 'float32', 'nombre_pieces_principales': 'float32',
                   'type_local': 'category', 'latitude': 'float32', 'longitude': 'float32'}

def load_specific_cities_data(city_codes, file_paths=FILE_PATHS_CSV, use_cols=COLS_FOR_CITY, dtypes=DTYPES_FOR_CITY):
    """ Charge et nettoie les données pour une liste de codes communes spécifiques. """
    print(f"🔬 Chargement des données pour les communes : {city_codes}...")
    all_city_chunks = []
    if not file_paths: print("❌ Aucun fichier CSV source trouvé."); return pd.DataFrame()
    city_codes_str = [str(c) for c in city_codes]

    for file_path in tqdm(file_paths, desc="Lecture CSV", leave=False):
        try:
            chunk_iter = pd.read_csv(file_path, usecols=lambda col: col in use_cols, dtype=dtypes,
                                     chunksize=100_000, low_memory=False, on_bad_lines='skip')
            for chunk in chunk_iter:
                if 'code_commune' in chunk.columns: # Vérifier si la colonne existe dans le chunk
                    chunk_filtered = chunk[chunk['code_commune'].astype(str).isin(city_codes_str)]
                    if not chunk_filtered.empty: all_city_chunks.append(chunk_filtered)
        except FileNotFoundError: print(f"\n⚠️ Fichier ignoré: {file_path}")
        except Exception as e: print(f"\n⚠️ Erreur lecture {file_path}: {e}")

    if not all_city_chunks: print("❌ Aucune donnée trouvée."); return pd.DataFrame()

    print("   -> Concaténation...")
    df_cities_raw = pd.concat(all_city_chunks, ignore_index=True)
    print(f"   -> {len(df_cities_raw):,} transactions brutes chargées.")
    df_cities_clean = clean_dvf_data(df_cities_raw)

    print("   -> Optimisation mémoire finale...")
    for col in df_cities_clean.select_dtypes(include=['object', 'category']).columns:
         if col not in ['trimestre']:
            df_cities_clean[col] = df_cities_clean[col].astype('category')
    gc.collect()
    print(f"✅ Chargement et nettoyage terminés pour {city_codes}.")
    # df_cities_clean.info(memory_usage='deep') # Optionnel, peut être verbeux
    return df_cities_clean

# Cache simple pour éviter de recharger les mêmes données ville si on relance la cellule d'estimation
COMMUNE_DATA_CACHE = {}

print("✅ Fonctions utilitaires définies.")

✅ Fonctions utilitaires définies.


 <div style="background: linear-gradient(to right, #fff3e0, #ffe0b2); /* Soft orange gradient */
             border-left: 4px solid #ffcc80; /* Lighter orange border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-orange h2 { color: #ef6c00; margin-bottom: 12px; } .custom-div-orange p { line-height: 1.6; } </style>
 <div class="custom-div-orange">

 ## 📈 2. Modélisation de la Rentabilité Locative (Besoin III.2)

 **Objectif :** Définir les fonctions pour estimer le loyer potentiel et calculer les rendements brut et net.

 <div class="assumption-note">
 ⚠️ Hypothèse Simplificatrice Importante : Ne disposant pas de source de données de loyers externes fiable et gratuite, nous avons simulé le loyer mensuel par m² comme un pourcentage du prix de vente médian par m² observé dans les données DVF pour des biens similaires. C'est une approximation forte, mais elle permet d'illustrer le calcul de rentabilité. Le pourcentage utilisé (basé sur le rendement brut annuel souhaité) sera un paramètre ajustable. 
 </div>

 **Fonctions :**
 * `simulate_rent_m2(median_price_m2, gross_yield_assumption)`: Calcule un loyer mensuel/m² simulé.
 * `calculate_yields(...)`: Calcule le rendement brut et net annuel à partir du loyer, du prix d'achat et des différentes charges (paramétrables).

 </div>
 </div>

In [3]:
def simulate_rent_m2(median_price_m2, gross_yield_assumption_percent=5.0):
    """
    SIMULE un loyer mensuel par m² basé sur le prix de vente médian/m²
    et une hypothèse de rendement brut annuel.

    Args:
        median_price_m2 (float): Prix de vente médian au m² estimé pour des biens similaires.
        gross_yield_assumption_percent (float): Hypothèse de rendement brut annuel souhaité (en %).

    Returns:
        float: Loyer mensuel par m² simulé, ou 0 si l'entrée est invalide.
    """
    if median_price_m2 <= 0 or gross_yield_assumption_percent <= 0:
        return 0
    # Loyer annuel total = Prix * RendementBrutAnnuel
    # Loyer annuel / m² = Prix_m2 * RendementBrutAnnuel
    # Loyer mensuel / m² = (Prix_m2 * RendementBrutAnnuel / 100) / 12
    rent_m2_monthly = (median_price_m2 * gross_yield_assumption_percent / 100) / 12
    return rent_m2_monthly

def calculate_yields(purchase_price, surface_m2, rent_m2_monthly,
                     other_charges_percent_rent=15.0, # Charges non récupérables (syndic, etc.) en % du loyer annuel HC
                     property_tax_percent_rent=8.0,   # Taxe foncière en % du loyer annuel HC (approximation très variable)
                     vacancy_percent_rent=5.0,        # Hypothèse de vacance locative en % du loyer annuel HC (ex: ~1/2 mois)
                     management_percent_rent=7.0,     # Frais de gestion locative en % du loyer annuel HC
                     insurance_gli_percent_rent=3.0,  # Assurance loyers impayés (GLI) en % du loyer annuel HC
                     minor_repairs_percent_rent=2.0): # Provision petites réparations/entretien en % du loyer annuel HC
    """
    Calcule les rendements locatifs brut et net annuels.

    Args:
        purchase_price (float): Prix d'achat total estimé (FAI).
        surface_m2 (float): Surface du bien en m².
        rent_m2_monthly (float): Loyer mensuel par m² (simulé ou réel).
        other_charges_percent_rent (float): Autres charges non récupérables (% loyer annuel).
        property_tax_percent_rent (float): Taxe foncière (% loyer annuel).
        vacancy_percent_rent (float): Taux de vacance (% loyer annuel).
        management_percent_rent (float): Frais de gestion (% loyer annuel).
        insurance_gli_percent_rent (float): Assurance GLI (% loyer annuel).
        minor_repairs_percent_rent (float): Provision réparations (% loyer annuel).

    Returns:
        dict: Dictionnaire contenant 'rendement_brut_annuel_pct' et 'rendement_net_annuel_pct'.
              Retourne des rendements de 0 si les entrées sont invalides.
    """
    if purchase_price <= 0 or surface_m2 <= 0 or rent_m2_monthly <= 0:
        return {'rendement_brut_annuel_pct': 0, 'rendement_net_annuel_pct': 0, 'loyer_annuel_hc': 0, 'charges_annuelles_totales': 0}

    # Loyer annuel Hors Charges (HC)
    annual_rent_hc = rent_m2_monthly * surface_m2 * 12

    # Calcul du rendement brut
    gross_yield_pct = (annual_rent_hc / purchase_price) * 100

    # Calcul des charges annuelles totales (basées sur le loyer annuel HC)
    total_charges_annual = (annual_rent_hc * (
        other_charges_percent_rent +
        property_tax_percent_rent +
        vacancy_percent_rent + # La vacance est une perte de revenu, assimilée à une charge ici pour le calcul net
        management_percent_rent +
        insurance_gli_percent_rent +
        minor_repairs_percent_rent
    ) / 100)

    # Revenu net annuel (avant impôts sur le revenu et prélèvements sociaux)
    net_income_annual = annual_rent_hc - total_charges_annual

    # Calcul du rendement net (avant IR et PS)
    # Note: On l'appelle souvent "net de charges" ou "net net" pour le distinguer du net fiscal.
    net_yield_pct = (net_income_annual / purchase_price) * 100 if purchase_price > 0 else 0

    return {
        'rendement_brut_annuel_pct': round(gross_yield_pct, 2),
        'rendement_net_annuel_pct': round(net_yield_pct, 2),
        'loyer_annuel_hc': round(annual_rent_hc, 0),
        'charges_annuelles_totales': round(total_charges_annual, 0)
    }

print("✅ Fonctions de simulation de loyer et de calcul de rendement définies.")

✅ Fonctions de simulation de loyer et de calcul de rendement définies.


<div style="background: linear-gradient(to right, #f3e5f5, #fce4ec); /* Soft purple/pink gradient */
             border-left: 4px solid #ce93d8; /* Lighter, thinner purple border */
             padding: 18px 20px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
             margin-bottom: 20px; animation: slideIn 0.7s ease-out;">
 <style> @keyframes slideIn { 0% { opacity: 0; transform: translateX(-10px); } 100% { opacity: 1; transform: translateX(0); } }
   .custom-div-purple h2 { color: #7b1fa2; margin-bottom: 12px; } .custom-div-purple p { line-height: 1.6; } </style>
 <div class="custom-div-purple">

 ## 🛠️ 3. Estimateur Interactif de Valeur et Rendement (Besoin II.1 & III.2)

 **Objectif :** Fournir à Alexandre un outil simple pour obtenir une première estimation pour un bien qui l'intéresse.

 **Fonctionnement :**
 1.  **Entrer les caractéristiques** du bien cible (commune, type, surface, pièces, état général).
 2.  **Entrer les hypothèses** financières (rendement brut visé, % de charges diverses).
 3.  **Cliquer sur "Estimer"**.
 4.  Le système va :
     * Charger les données DVF pour la commune spécifiée (via `load_specific_cities_data`).
     * Filtrer ces données pour trouver des transactions **comparables** (même type, surface et nb pièces proches, sur les 2 dernières années).
     * Calculer le **prix médian/m²** de ces comparables.
     * Estimer la **valeur du bien cible** en appliquant un facteur d'état (neuf, bon état, à rénover).
     **Simuler le loyer** basé sur le prix/m² estimé et le rendement brut visé.
     * Calculer les **rendements brut et net** simulés avec les charges fournies.
     * Afficher un **résumé** et permettre d'**exporter les données comparables** utilisées.

 </div>
 </div>

In [6]:
def estimate_property_and_yield(
    code_commune,
    prop_type='Appartement',
    surface=50.0,
    nb_pieces=2,
    etat_general=1.0, # Facteur: 1.1=Neuf/Refait, 1.0=Bon état, 0.9=Travaux à prévoir, 0.8=Gros travaux
    gross_yield_target=5.0,
    charges_pct=15.0,
    taxe_fonciere_pct=8.0,
    vacance_pct=5.0,
    gestion_pct=7.0,
    gli_pct=3.0,
    reparations_pct=2.0,
    years_comparable=2 # Analyser les X dernières années pour les comparables
    ):
    """
    Fonction principale pour estimer la valeur et la rentabilité simulée d'un bien.
    """
    global COMMUNE_DATA_CACHE # Utiliser le cache global

    print(f"\n--- Lancement de l'estimation pour {code_commune} ---")
    start_time = pd.Timestamp.now()

    # --- 1. Chargement/Cache des données de la commune ---
    code_commune_str = str(code_commune).strip()
    if not code_commune_str:
        print("❌ Code commune invalide.")
        return None, pd.DataFrame(), "Code commune manquant."

    if code_commune_str in COMMUNE_DATA_CACHE:
        print(f"Utilisation des données en cache pour {code_commune_str}.")
        df_city = COMMUNE_DATA_CACHE[code_commune_str]
    else:
        df_city = load_specific_cities_data([code_commune_str])
        if df_city.empty:
            return None, pd.DataFrame(), f"Aucune donnée DVF trouvée ou valide pour la commune {code_commune_str}."
        COMMUNE_DATA_CACHE[code_commune_str] = df_city # Mettre en cache

    if df_city.empty: # Vérification double après le cache potentiel
         return None, pd.DataFrame(), f"Données vides pour la commune {code_commune_str}."

    nom_commune = df_city['nom_commune'].iloc[0]
    latest_year = df_city['annee'].max()
    start_year_comp = latest_year - years_comparable + 1

    # --- 2. Filtrage des comparables ---
    print(f"Recherche de comparables ({prop_type}, {nb_pieces}p, ~{surface}m², état ~{etat_general:.1f}) sur {start_year_comp}-{latest_year}...")

    # Tolérances pour surface et pièces
    surface_min = surface * 0.80
    surface_max = surface * 1.20
    pieces_min = max(1, nb_pieces - 1)
    pieces_max = nb_pieces + 1

    df_comparables = df_city[
        (df_city['type_local'] == prop_type) &
        (df_city['annee'] >= start_year_comp) &
        (df_city['surface_reelle_bati'].between(surface_min, surface_max)) &
        (df_city['nombre_pieces_principales'].between(pieces_min, pieces_max))
    ]

    if len(df_comparables) < 5: # Seuil minimum pour une estimation fiable
        print(f"⚠️ Moins de 5 comparables trouvés ({len(df_comparables)}). L'estimation peut être imprécise.")
        if len(df_comparables) == 0:
             return None, df_comparables, f"Aucun comparable trouvé pour ces critères à {nom_commune}."
    else:
        print(f"   -> {len(df_comparables)} comparables trouvés.")

    # --- 3. Estimation de la valeur ---
    median_price_m2_comp = df_comparables['prix_m2'].median()
    estimated_price_m2_target = median_price_m2_comp * etat_general # Ajustement selon l'état
    estimated_purchase_price = estimated_price_m2_target * surface

    print(f"   -> Prix/m² médian des comparables : {median_price_m2_comp:,.0f} EUR")
    print(f"   -> Prix/m² estimé pour le bien cible (état {etat_general:.1f}) : {estimated_price_m2_target:,.0f} EUR")
    print(f"   -> Valeur estimée du bien cible ({surface} m²) : {estimated_purchase_price:,.0f} EUR")

    # --- 4. Simulation Loyer & Rendements ---
    simulated_rent_m2 = simulate_rent_m2(estimated_price_m2_target, gross_yield_target)
    if simulated_rent_m2 == 0:
        print("❌ Impossible de simuler le loyer (prix/m² ou rendement cible invalide).")
        yield_results = {'rendement_brut_annuel_pct': 0, 'rendement_net_annuel_pct': 0, 'loyer_annuel_hc': 0, 'charges_annuelles_totales': 0}
    else:
        print(f"   -> Loyer mensuel simulé basé sur {gross_yield_target:.1f}% brut : {simulated_rent_m2:,.2f} EUR/m²")
        yield_results = calculate_yields(
            purchase_price=estimated_purchase_price,
            surface_m2=surface,
            rent_m2_monthly=simulated_rent_m2,
            other_charges_percent_rent=charges_pct,
            property_tax_percent_rent=taxe_fonciere_pct,
            vacancy_percent_rent=vacance_pct,
            management_percent_rent=gestion_pct,
            insurance_gli_percent_rent=gli_pct,
            minor_repairs_percent_rent=reparations_pct
        )

    # --- 5. Préparation du résultat ---
    results = {
        'Commune': f"{nom_commune} ({code_commune_str})",
        'Type Bien': prop_type,
        'Surface (m²)': surface,
        'Nb Pièces': nb_pieces,
        'Facteur État': etat_general,
        'Nb Comparables': len(df_comparables),
        'Prix/m² Comparables': round(median_price_m2_comp, 0),
        'Prix/m² Cible Estimé': round(estimated_price_m2_target, 0),
        'Valeur Cible Estimée': round(estimated_purchase_price, 0),
        'Hyp. Rend. Brut (%)': gross_yield_target,
        'Loyer Mensuel Simulé (/m²)': round(simulated_rent_m2, 2),
        'Loyer Annuel HC Simulé': yield_results['loyer_annuel_hc'],
        'Charges Annuelles Estimées': yield_results['charges_annuelles_totales'],
        'Rendement Brut Simulé (%)': yield_results['rendement_brut_annuel_pct'],
        'Rendement Net Simulé (%)': yield_results['rendement_net_annuel_pct']
    }

    elapsed = (pd.Timestamp.now() - start_time).total_seconds()
    print(f"✅ Estimation terminée en {elapsed:.2f} secondes.")

    return results, df_comparables, None # None = pas de message d'erreur

print("✅ Fonction d'estimation définie.")

# %%
# --- Création des Widgets pour l'Estimateur ---

style = {'description_width': '180px'}
layout = widgets.Layout(width='400px')
layout_wide = widgets.Layout(width='600px')

# --- Inputs Bien Cible ---
w_code_commune = widgets.Text(description="Code Commune (ex: 69100):", style=style, layout=layout)
w_prop_type = widgets.Dropdown(options=['Appartement', 'Maison'], value='Appartement', description="Type de Bien:", style=style, layout=layout)
w_surface = widgets.FloatSlider(min=10, max=250, step=5, value=50, description="Surface (m²):", style=style, layout=layout_wide, readout_format='.0f')
w_nb_pieces = widgets.IntSlider(min=1, max=10, step=1, value=2, description="Nombre de Pièces:", style=style, layout=layout_wide)
w_etat = widgets.Dropdown(
    options=[('Neuf / Refait à neuf', 1.1), ('Bon état / Standard', 1.0), ('Travaux à prévoir', 0.9), ('Gros travaux / À rénover', 0.8)],
    value=1.0, description="État Général:", style=style, layout=layout
)

# --- Inputs Hypothèses Financières ---
w_gross_yield = widgets.FloatSlider(min=1.0, max=15.0, step=0.5, value=5.0, description="Rendement Brut Annuel Visé (%):", style=style, layout=layout_wide, readout_format='.1f')
w_charges = widgets.FloatSlider(min=0.0, max=50.0, step=1.0, value=15.0, description="Charges non récup. (% loyer):", style=style, layout=layout_wide, readout_format='.0f')
w_taxe_fonciere = widgets.FloatSlider(min=0.0, max=30.0, step=1.0, value=8.0, description="Taxe Foncière (% loyer):", style=style, layout=layout_wide, readout_format='.0f')
w_vacance = widgets.FloatSlider(min=0.0, max=25.0, step=1.0, value=5.0, description="Vacance Locative (% loyer):", style=style, layout=layout_wide, readout_format='.0f')
w_gestion = widgets.FloatSlider(min=0.0, max=15.0, step=0.5, value=7.0, description="Frais de Gestion (% loyer):", style=style, layout=layout_wide, readout_format='.1f')
w_gli = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value=3.0, description="Assurance GLI (% loyer):", style=style, layout=layout_wide, readout_format='.1f')
w_reparations = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value=2.0, description="Provision Réparations (% loyer):", style=style, layout=layout_wide, readout_format='.1f')

# --- Bouton et Zone de Résultat ---
w_button_estimate = widgets.Button(description="✅ Estimer Valeur & Rendement", button_style='success', icon='calculator', layout=widgets.Layout(width='300px', height='40px'))
w_output_area = widgets.Output()
w_export_button_placeholder = widgets.Output() # Placeholder pour le bouton d'export qui apparaitra après

# --- Organisation des Widgets ---
box_bien = widgets.VBox([w_code_commune, w_prop_type, w_surface, w_nb_pieces, w_etat], layout=widgets.Layout(border='1px solid #ce93d8', padding='10px', margin='5px'))
box_finances = widgets.VBox([w_gross_yield, w_charges, w_taxe_fonciere, w_vacance, w_gestion, w_gli, w_reparations], layout=widgets.Layout(border='1px solid #a5d6a7', padding='10px', margin='5px'))

ui = widgets.VBox([
    widgets.HTML("<h2>Entrez les caractéristiques du bien cible et vos hypothèses :</h2>"),
    widgets.HBox([box_bien, box_finances]),
    w_button_estimate,
    widgets.HTML("<hr><h2>Résultats de l'Estimation :</h2>"),
    w_output_area,
    w_export_button_placeholder
])

# Variable globale pour stocker les données comparables pour l'export
comparables_data_for_export = pd.DataFrame()

print("✅ Widgets de l'estimateur créés.")

# %%
# --- Logique du Bouton d'Estimation ---

def create_download_link_df(df, title, filename):
    """ Crée un lien de téléchargement HTML pour un DataFrame (CSV). """
    csv = df.to_csv(index=False, encoding='utf-8-sig')
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:text/csv;base64,{b64}" download="{filename}" target="_blank">{title}</a>'
    return href

def on_estimate_button_clicked(b):
    global comparables_data_for_export # Modifier la variable globale

    with w_output_area:
        clear_output(wait=True) # Effacer les anciens résultats
        w_export_button_placeholder.clear_output() # Effacer l'ancien bouton d'export
        print("🚀 Lancement de l'estimation...")

        # Récupérer les valeurs des widgets
        code = w_code_commune.value
        ptype = w_prop_type.value
        surf = w_surface.value
        pieces = w_nb_pieces.value
        etat = w_etat.value
        gy = w_gross_yield.value
        ch = w_charges.value
        tf = w_taxe_fonciere.value
        vac = w_vacance.value
        gest = w_gestion.value
        gli = w_gli.value
        rep = w_reparations.value

        # Appeler la fonction d'estimation
        results, df_comp, error_msg = estimate_property_and_yield(
            code_commune=code, prop_type=ptype, surface=surf, nb_pieces=pieces, etat_general=etat,
            gross_yield_target=gy, charges_pct=ch, taxe_fonciere_pct=tf, vacance_pct=vac,
            gestion_pct=gest, gli_pct=gli, reparations_pct=rep
        )

        # Afficher le résultat ou l'erreur
        if error_msg:
            print(f"❌ Erreur : {error_msg}")
            comparables_data_for_export = pd.DataFrame() # Réinitialiser
        elif results:
            # Affichage formaté des résultats
            html_output = "<h3>Synthèse de l'Estimation :</h3><table border='1' style='border-collapse: collapse; width: 100%;'>"
            for key, value in results.items():
                # Formatage conditionnel pour les nombres
                if isinstance(value, (int, float)):
                    if '%' in key:
                        formatted_value = f"{value:.2f}%"
                    elif 'Prix' in key or 'Valeur' in key or 'Loyer' in key or 'Charges' in key:
                         formatted_value = f"{value:,.0f} EUR"
                    elif 'Surface' in key:
                        formatted_value = f"{value:.0f} m²"
                    elif 'Facteur' in key:
                        formatted_value = f"{value:.1f}"
                    else:
                        formatted_value = f"{value:,}"
                else:
                    formatted_value = value
                html_output += f"<tr><td style='padding: 5px;'><strong>{key}</strong></td><td style='padding: 5px;'>{formatted_value}</td></tr>"
            html_output += "</table>"

            display(HTML(html_output))

            # Stocker les données pour l'export et afficher le bouton
            comparables_data_for_export = df_comp
            if not comparables_data_for_export.empty:
                export_title = f"📥 Exporter les {len(comparables_data_for_export)} comparables (.csv)"
                export_filename = f"comparables_{code}_{datetime.now().strftime('%Y%m%d%H%M')}.csv"
                download_link = create_download_link_df(comparables_data_for_export, export_title, export_filename)
                with w_export_button_placeholder: # Afficher dans le placeholder
                    display(HTML(f"<br>{download_link}"))
            else:
                 comparables_data_for_export = pd.DataFrame() # S'assurer qu'il est vide si aucun comp n'est trouvé

        else:
            print("❌ Une erreur inattendue s'est produite lors de l'estimation.")
            comparables_data_for_export = pd.DataFrame() # Réinitialiser


# Lier la fonction au clic du bouton
w_button_estimate.on_click(on_estimate_button_clicked)

# Afficher l'interface utilisateur complète
display(ui)

print("\n✅ Interface de l'estimateur prête. Entrez les informations et cliquez sur 'Estimer'.")

✅ Fonction d'estimation définie.
✅ Widgets de l'estimateur créés.



✅ Interface de l'estimateur prête. Entrez les informations et cliquez sur 'Estimer'.
